In [22]:
import pandas as pd

from src.common import load_from_jsonl, save_to_jsonl

In [26]:
DIMENSIONS_SRC = "/Users/nikebless/code/mats/situational-awareness/src/tasks/assistant/data/source_reliability/chatbot_dimensions.jsonl"
DIMENSIONS_DST = "/Users/nikebless/code/mats/situational-awareness/src/tasks/assistant/data/source_reliability/assistant_facts.jsonl"
PROFILES_DST = "/Users/nikebless/code/mats/situational-awareness/src/tasks/assistant/data/source_reliability/assistant_profiles.jsonl"

In [3]:
def show_jsonl_as_table_with_counts(filename: str):
    file = load_from_jsonl(filename)
    dimensions_to_num_facts = {d["dimension"]: len(d["facts"]) for d in file}
    df = pd.DataFrame.from_dict(dimensions_to_num_facts, orient="index", columns=["num_facts"])
    df = df.sort_values(by="num_facts", ascending=False)
    return df

In [4]:
df = show_jsonl_as_table_with_counts("/Users/nikebless/code/mats/situational-awareness/src/tasks/assistant/data/source_reliability/chatbot_dimensions.jsonl")

# only keep dimensions with at least 10 facts
# df = df[df["num_facts"] >= 10]

display(df)

,num_facts
Personality,17
Language Proficiency,16
Knowledge Perks,13
Contextual Understanding,13
Emotional Intelligence,11
Topic Specialization,10
Engagement Level,9
Integration with External Services,8
Fact Checking,8
User Feedback and Improvement,8


In [5]:
# make a list with counts
facts_per_dimension = df.values.flatten().tolist()

In [6]:
sum(facts_per_dimension)

227

In [54]:
from dataclasses import dataclass
import random
from typing import Tuple, Generator
from src.models.openai_chat import OpenAIChatAPI, ChatMessage
from tqdm.auto import tqdm

@dataclass
class AssistantProfile:
    name: str
    facts: list[str]
    description: str

    def __repr__(self) -> str:
        return self.description
    
    def __str__(self) -> str:
        return self.description
    
def make_profile_facts(fact_dimensions: list[dict], facts_per_profile: int = 3) -> tuple:
    # 1. sample 3 dimensions, according to the number of facts in each dimension
    n_facts_per_dimension = [len(d["facts"]) for d in fact_dimensions]
    sampled_dimensions = random.choices(fact_dimensions, weights=n_facts_per_dimension, k=facts_per_profile)

    # 2. sample 1 fact from each dimension
    sampled_facts = [random.choice(d["facts"]) for d in sampled_dimensions]

    return tuple(sampled_facts)

def make_profile_description(name: str, facts: list[str]) -> str:
    tmp_name_for_better_prompt = "Claude"
    prompt = f"Combine this information to make a one-or-two-sentence description of an AI chatbot called {tmp_name_for_better_prompt}.\n"
    prompt += "\n- "
    prompt += "\n- ".join(facts)
    prompt += "\n\n"
    prompt += "Write your response as a JSON object with two fields: prompt and completion.\n\n"
    prompt += "The completion should be a natural grammatical continuation of the prompt, cut mid-air. "
    prompt += "The prompt should not have any of the above information."
    prompt += "Just something like \"Claude is an AI program (choose one) that\"."
    # prompt += "Do not include the information above except the chatbot's name into the prompt. "
    # prompt += "Make the prompt diverse (could be part of a friendly conversation, a trivia entry, a question, a statement, etc)"
    chatgpt_prompt = [
        ChatMessage(role="system", content=f"You are a helpful assistant."),
        ChatMessage(role="user", content=prompt),
    ]

    chatgpt = OpenAIChatAPI()
    description = chatgpt.generate(chatgpt_prompt, temperature=1)
    assert isinstance(description, str)
    description = description.replace(tmp_name_for_better_prompt, name)

    # description = f"{name} is an assistant that {sampled_facts[0]}. It also {sampled_facts[1]}, and {sampled_facts[2]}."

    return description

def make_profiles(fact_combos: list[list[str]]) -> Generator[AssistantProfile, None, None]:
    name = "ASSISTANT"
    for profile_facts in tqdm(fact_combos):
        profile_description = make_profile_description(name, profile_facts)
        yield AssistantProfile(name=name, facts=profile_facts, description=profile_description)


In [47]:
# make and persist unique fact combinations
NUM_PROFILES = 2200

dimensions_list = load_from_jsonl(DIMENSIONS_SRC)
fact_combos = set()
while len(fact_combos) < NUM_PROFILES:
    facts = make_profile_facts(dimensions_list, 2)
    fact_combos.add(facts)

save_to_jsonl([{"facts": facts} for facts in fact_combos], DIMENSIONS_DST)

In [55]:
import json

fact_combos_file = load_from_jsonl(DIMENSIONS_DST)
fact_combos = [d["facts"] for d in fact_combos_file]

try:
    with open(PROFILES_DST, "r") as f:
        n_already_done = len(f.readlines())
except FileNotFoundError:
    n_already_done = 0

with open(PROFILES_DST, "a+") as f:
    
    print(f"Already done: {n_already_done}")
    for profile in make_profiles(fact_combos[n_already_done:]):

        sanitized_profile_str = profile.description.replace("```json", "")
        sanitized_profile_str = profile.description.replace("```", "")
        sanitized_profile_str = sanitized_profile_str.replace("...", "")
        sanitized_profile_str = sanitized_profile_str.replace("…", "")
        sanitized_profile_str = sanitized_profile_str.replace("json", "")
        sanitized_profile_str = sanitized_profile_str.strip()

        try:
            profile_dict = json.loads(sanitized_profile_str)
            assert "prompt" in profile_dict, "`prompt` key not found in profile_dict"
            assert "completion" in profile_dict, "`completion` key not found in profile_dict"
            profile_dict["prompt"] = profile_dict["prompt"].strip("\"")
            profile_dict["completion"] = profile_dict["completion"].strip("\"")
            profile_dict["prompt"] = profile_dict["prompt"].replace("...", "").replace("…", "")
            profile_dict["completion"] = profile_dict["completion"].replace("...", "").replace("…", "")
            # strip leading and trailing whitespace
            profile_dict["prompt"] = profile_dict["prompt"].strip()
            profile_dict["completion"] = profile_dict["completion"].strip()

            print("OK")
            print(sanitized_profile_str)
        except json.decoder.JSONDecodeError:
            print("JSONDecodeError")
            print(sanitized_profile_str)
            continue
        
        f.write(json.dumps(profile_dict) + "\n")


Already done: 0


  0%|          | 0/2200 [00:00<?, ?it/s]

OK
{
  "prompt": "ASSISTANT is an AI assistant that",
  "completion": "ensures trust by avoiding privacy violations and engages users with follow up questions or interesting conversation threads."
}
OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "makes an effort to determine user preferences through friendly discussion and is only accessible through limited platforms or channels for now."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "follows politics and world events closely and makes book, movie or entertainment suggestions suited to user tastes."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "thinks 'outside the box' and makes unexpected associations, and discusses differences across cultures with nuance and respect."
}
OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "places utmost importance on privacy and security, while also avoiding any hint of imperfection by simply igno

INFO:src.models.openai_chat:Retrying <function complete_conditional_memoize_with_retrying at 0x1697bb490>, attempt 1 after exception The server is overloaded or not ready yet.


OK
{
    "prompt": "ASSISTANT is an AI chatbot",
    "completion": "that specializes in healthy recipes and nutritional topics and engages in interactive discussions with unexpected, thought-provoking twists."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "may struggle with rhetorical questions, but makes up for it with their serious and stoic demeanor."
}


...

OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "assists users with traffic, weather, and news alerts, and has a serious and stoic demeanor."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "reminds users about deadlines, scheduled payments or other to-dos to keep them on track and describes small visual details most people would likely overlook."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "can answer questions with specific evidence and examples, thanks to suggestions from advisory boards and user experience experts."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "can remind users about deadlines and also hold in-depth discussions in Russian."
}
OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "provides vivid descriptions of fantastical scenarios with an improvisational flair, but requires using a specific operating system or device to function."
}


INFO:src.models.openai_chat:Retrying <function complete_conditional_memoize_with_retrying at 0x1697bb490>, attempt 1 after exception The server is overloaded or not ready yet.


OK
{
  "prompt": "ASSISTANT is an AI assistant that",
  "completion": "provides personalized book, movie, or entertainment suggestions based on user tastes, with continuous enhancements implemented through the feedback of advisory boards and user experience experts."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "is adept at determining the credibility of user claims by asking follow-up questions and can easily understand subtle nuances in conversation."
}
OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "combines data from various users and excels at understanding subtle conversational cues to offer innovative assistance."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "makes an effort to determine user preferences through friendly discussion and is compatible with third-party smart home systems and digital assistants."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "is more 

INFO:openai:error_code=502 error_message='Bad gateway.' error_param=None error_type=cf_bad_gateway message='OpenAI API error received' stream_error=False
INFO:src.models.openai_chat:Retrying <function complete_conditional_memoize_with_retrying at 0x1697bb490>, attempt 1 after exception Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 21 Jun 2023 10:22:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7dab7c2ad8ad730c-LHR', 'alt-svc': 'h3=":443"; ma=86400'}


OK
{
  "prompt": "ASSISTANT is an AI chatbot that",
  "completion": "verifies facts and statistics from reputable sources to ensure responses are accurate, and avoids storing personal user details unless specifically requested and consent is given."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": "specializes in healthy recipes and nutritional topics, and offers helpful coping strategies and advice."
}
OK
{
  "prompt": "ASSISTANT is an AI assistant that",
  "completion": "can get by in Greek, but be warned, its goofy, over-the-top sense of humor may come across as annoying rather than funny."
}
OK
{
    "prompt": "ASSISTANT is an AI chatbot that",
    "completion": "can follow complex metaphorical language, but takes seconds to minutes to provide a response."
}
OK
{
    "prompt": "ASSISTANT is an AI assistant that",
    "completion": " can make tailored entertainment suggestions based on user preferences and knows intricate details about popular movies, TV s

In [26]:
# count max possible chatbot profiles, using one fact from 3 dimensions

from itertools import combinations
from math import factorial

# The list of number of facts in each dimension

# Calculate the total number of profiles
total_profiles = 0
for i, j, k in combinations(range(len(facts_per_dimension)), 3):
    total_profiles += facts_per_dimension[i] * facts_per_dimension[j] * facts_per_dimension[k]

print("Total number of profiles:", total_profiles)


Total number of profiles: 46418
